In [1]:
qr(matrix(c(1,0.5,0.5,1),2))

$qr
           [,1]       [,2]
[1,] -1.1180340 -0.8944272
[2,]  0.4472136  0.6708204

$rank
[1] 2

$qraux
[1] 1.8944272 0.6708204

$pivot
[1] 1 2

attr(,"class")
[1] "qr"

In [2]:
eigen(matrix(c(1,0.5,0.5,1),2))

0.7071068,-0.7071068
0.7071068,0.7071068


# 谱分解的 Jacobi 算法

容易想到，对于实对称矩阵而言，高斯消元法那个套路若同时伴随对列也做，就可以变换出一个对角矩阵。

$$
B_m \cdots B_2 B_1 A B_1^T B_2^T \cdots B_m^T = \Lambda
$$

这暗示了我们已经获取了分解
$$
A = (B_m \cdots B_1)^{-1} \Lambda ((B_m \cdots B_1)^T)^{-1} 
$$

虽然可以写成这样的分解，但没有任何信息显示了$(B_m \cdots B_1)^{-1}$是特征向量构成的矩阵或为正交矩阵（显然一般不是）。或$\Lambda$是特征值矩阵。
这和我们想要的那种$P\Lambda P^T$还有点距离。

这里的$B_i$本来是高斯消元法中使用的初等矩阵。我们转而使用Jacobi变换矩阵，记为$G_{ij}$，记号$G$暗示它是Givens变换阵的一员。

In [56]:
jac.param <- function(A,i,j){
    n <- nrow(A)
    x <- 2*A[i,j]
    y <- A[i,i] - A[j,j]
    c <- sqrt(0.5*(1+y/(sqrt(x^2+y^2))))
    d <- x/(2*c*sqrt(x^2+y^2))
    
    return(list(i = i,j = j,c = c,d = d,n = n))
}

jac.trans <- function(param,A){
    i = param$i
    j = param$j
    c = param$c
    d = param$d
    n = param$n
    
    a = A
    aS = A
    
    t <- (1:n)[-c(i,j)]
    aS[i,t] <- aS[t,i] <- c * a[i,t] + d * a[j,t]
    aS[j,t] <- aS[t,j] <- c * a[j,t] - d * a[i,t]
    aS[i,i] <- c^2*a[i,i] + d^2*a[j,j] + 2*c*d*a[i,j]
    aS[j,j] <- d^2*a[i,i] + c^2*a[j,j] - 2*c*d*a[i,j]
    aS[i,j] <- aS[j,i] <- (a[j,j] - a[i,i])*c*d + a[i,j]*(c^2 - d^2)
    
    return(aS)

}

In [17]:
rcov <- function(n, r, Lam=NULL){
    if(is.null(Lam)){
        lam <- rep(0,n)
        lam[1:r] <- runif(r,0.5,1.5)
        Lam <- diag(lam)
    }
    A <- matrix(runif(n*n,-2,2),n)
    return(t(A) %*% Lam %*% A)
}


In [42]:
Sigma <- rcov(5,5)
Sigma

7.291794,2.153550,2.065414,3.681454,-3.416670
2.153550,3.153542,-3.465860,2.787932,1.558811
2.065414,-3.465860,9.307293,-1.970986,-5.099413
3.681454,2.787932,-1.970986,5.262893,2.442923
-3.416670,1.558811,-5.099413,2.442923,7.023584


In [77]:
jac.trans(jac.param(Sigma,1,3),Sigma)

1.059769e+01,-1.798280,-1.110223e-16,0.2790727,-6.1350920
-1.798280e+00,3.153542,-3.662806e+00,2.7879316,1.5588114
-1.110223e-16,-3.662806,6.001394e+00,-4.1665338,0.1956787
2.790727e-01,2.787932,-4.166534e+00,5.2628930,2.4429228
-6.135092e+00,1.558811,1.956787e-01,2.4429228,7.0235835


In [33]:
jac.trans(Sigma,1,2)

ERROR: Error in a[j, j]: 量度数目不对


In [86]:
A <- Sigma
i <- 1
j <- 3
n <- 5

In [98]:
U <- diag(rep(1,5))

for(i in 1:(n-1)){
    for (j in (i+1):n){
        jac <- jac.param(A,i,j)
        A <- jac.trans(jac,A)
        U <- jac.trans(jac,U)
    }
}

In [99]:
A

1.623082e+01,2.428366e-66,5.058631e-72,2.440799e-105,2.131234e-126
2.428366e-66,1.156608e+01,9.150195e-82,4.414987e-115,-1.441800e-143
5.058631e-72,9.150195e-82,3.701357e+00,-1.710569e-49,3.335223e-127
2.440799e-105,4.414987e-115,-1.710569e-49,5.081179e-01,0.000000e+00
2.131234e-126,-1.441800e-143,3.335223e-127,0.000000e+00,3.272974e-02


In [91]:
A2 <- jac.trans(jac.param(A,1,2),A)
A2

1.621903e+01,-1.110223e-16,0.26251029,1.665880e-01,-2.694000e-01
-1.110223e-16,1.156267e+01,0.13815420,3.558641e-02,-9.995290e-02
2.625103e-01,1.381542e-01,3.70885545,2.846803e-02,-3.765068e-02
1.665880e-01,3.558641e-02,0.02846803,5.101735e-01,1.387779e-17
-2.694000e-01,-9.995290e-02,-0.03765068,1.387779e-17,3.838345e-02


In [85]:
A3 <- jac.trans(jac.param(A2,1,3),A2)
A3

9.529695e+00,-3.698706,2.220446e-16,-0.1227114,-5.6790765
-3.698706e+00,2.236189,-1.517894e+00,1.1221647,2.7731073
2.220446e-16,-1.517894,7.986746e+00,-4.8924571,0.4068201
-1.227114e-01,1.122165,-4.892457e+00,5.2628930,2.4429228
-5.679077e+00,2.773107,4.068201e-01,2.4429228,7.0235835


In [103]:
rep

function (x, ...)  .Primitive("rep")

In [105]:
c(Sigma)

[1]  7.291794  2.153550  2.065414  3.681454 -3.416670  2.153550  3.153542
 [8] -3.465860  2.787932  1.558811  2.065414 -3.465860  9.307293 -1.970986
[15] -5.099413  3.681454  2.787932 -1.970986  5.262893  2.442923 -3.416670
[22]  1.558811 -5.099413  2.442923  7.023584

In [106]:
Sigma

7.291794,2.153550,2.065414,3.681454,-3.416670
2.153550,3.153542,-3.465860,2.787932,1.558811
2.065414,-3.465860,9.307293,-1.970986,-5.099413
3.681454,2.787932,-1.970986,5.262893,2.442923
-3.416670,1.558811,-5.099413,2.442923,7.023584


In [109]:
rep(1,10)

[1] 1 1 1 1 1 1 1 1 1 1